In [0]:
# Databricks notebook source
# This notebook processes regional data based on the executing user's permissions

# Get parameters
dbutils.widgets.text("catalog", "security_demo", "Catalog")
dbutils.widgets.text("schema", "orders_data", "Schema")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

# Show current user context
print(f"Executing as: {spark.sql('SELECT CURRENT_USER()').collect()[0][0]}")
print(f"Compute Type: Serverless")

# Query 1: Count accessible records
customer_count = spark.sql(f'''
    SELECT COUNT(*) as count 
    FROM {catalog}.{schema}.customers
''').collect()[0][0]

order_count = 0
try:
    order_count = spark.sql(f'''
        SELECT COUNT(*) as count 
        FROM {catalog}.{schema}.orders
    ''').collect()[0][0]
except Exception as e:
    print(f"Note: Cannot access orders table - {str(e)[:50]}...")

print(f"Accessible customers: {customer_count}")
print(f"Accessible orders: {order_count}")

# Query 2: Show regional breakdown
regional_summary = spark.sql(f'''
    SELECT 
        region,
        COUNT(*) as customer_count,
        COUNT(DISTINCT customer_segment) as segments
    FROM {catalog}.{schema}.customers
    GROUP BY region
    ORDER BY region
''')

print("\\nRegional Summary:")
regional_summary.show()

# Query 3: Test column masking
masked_data = spark.sql(f'''
    SELECT 
        customer_name,
        email,
        phone,
        region
    FROM {catalog}.{schema}.customers
    LIMIT 3
''')

print("\\nSample Customer Data (with masking):")
masked_data.show(truncate=False)

# Return results as JSON for verification
import json
results = {
    "user": spark.sql('SELECT CURRENT_USER()').collect()[0][0],
    "customer_count": customer_count,
    "order_count": order_count,
    "regions": [row.region for row in regional_summary.collect()],
    "compute": "serverless"
}

dbutils.notebook.exit(json.dumps(results))